In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
! pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import keras_tuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 35s]
val_accuracy: 0.8738333582878113

Best val_accuracy So Far: 0.8880833387374878
Total elapsed time: 00h 26m 29s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 12s 7ms/step - loss: 0.5036 - accuracy: 0.8230 - val_loss: 0.4203 - val_accuracy: 0.8476
Epoch 2/50
1500/1500 [==============================] - 11s 7ms/step - loss: 0.3757 - accuracy: 0.8635 - val_loss: 0.3662 - val_accuracy: 0.8632
Epoch 3/50
1500/1500 [==============================] - 10s 7ms/step - loss: 0.3359 - accuracy: 0.8772 - val_loss: 0.3713 - val_accuracy: 0.8624
Epoch 4/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3086 - accuracy: 0.8851 - val_loss: 0.3503 - val_accuracy: 0.8691
Epoch 5/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2887 - accuracy: 0.8932 - val_loss: 0.3218 - val_accuracy: 0.8817
Epoch 6/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2726 - accuracy: 0.8990 - val_loss: 0.3321 - val_accuracy: 0.8793
Epoch 7/50
1500/1500 [==============================] - 11s 8ms/step - loss: 0.2587 - accuracy: 0.9039 - val_loss: 0.3284 - val_ac

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/35
1500/1500 [==============================] - 10s 6ms/step - loss: 0.5057 - accuracy: 0.8202 - val_loss: 0.4355 - val_accuracy: 0.8445
Epoch 2/35
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3745 - accuracy: 0.8634 - val_loss: 0.3845 - val_accuracy: 0.8607
Epoch 3/35
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3339 - accuracy: 0.8769 - val_loss: 0.3359 - val_accuracy: 0.8767
Epoch 4/35
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3080 - accuracy: 0.8860 - val_loss: 0.3299 - val_accuracy: 0.8824
Epoch 5/35
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2887 - accuracy: 0.8922 - val_loss: 0.3402 - val_accuracy: 0.8808
Epoch 6/35
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2756 - accuracy: 0.8975 - val_loss: 0.3292 - val_accuracy: 0.8802
Epoch 7/35
1500/1500 [==============================] - 9s 6ms/step - loss: 0.2622 - accuracy: 0.9023 - val_loss: 0.3176 - val_accuracy

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4850 - accuracy: 0.8862
[test loss, test accuracy]: [0.4850344657897949, 0.8862000107765198]
